In [1]:
from IPython.display import Markdown
import pandas as pd
import altair as alt
from fdp.db import query

_ = alt.themes.enable('powerbi')


In [2]:
storage_provider_power_df = query("select * from filecoin_storage_providers_power")


In [3]:
# | output: asis

print(
    f""" 
Updated on {storage_provider_power_df['date'].max().strftime('%Y-%m-%d')}
"""
)


 
Updated on 2024-01-16



In [4]:
latest_storage_provider_power_df = storage_provider_power_df[
    storage_provider_power_df["date"]
    == storage_provider_power_df["date"].max()
]

unique_miners_with_active_power = latest_storage_provider_power_df[
    latest_storage_provider_power_df["raw_power_pibs"] > 0
].nunique()["provider_id"]


::: {layout="[ [1,1,1]]"}

In [5]:
# | output: asis

print(
    f"""
### Active Storage Providers
{unique_miners_with_active_power}
"""
)



### Active Storage Providers
3135



In [6]:
# | output: asis

print(
    f"""
### Total Raw Power
{latest_storage_provider_power_df['raw_power_pibs'].sum():.2f} PiBs
"""
)



### Total Raw Power
8663.73 PiBs



In [7]:
# | output: asis

print(
    f"""
### Total Quality Adjusted PiB Power
{latest_storage_provider_power_df['quality_adjusted_power_pibs'].sum():.2f} PiBs
"""
)



### Total Quality Adjusted PiB Power
25571.03 PiBs



:::

::: {layout="[ [1,1], [1] ]"}


### Storage Provider Raw PiB Power

In [8]:
# | output: true

c = (
    alt.Chart(
        storage_provider_power_df.groupby("date")
        .sum()["raw_power_pibs"]
        .reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("raw_power_pibs:Q", title="Raw PiB Power"),
        tooltip=["date", "raw_power_pibs"],
    )
    .properties(width="container")
)

c.display()


alt.Chart(...)

### Storage Provider Quality Adjusted PiB Power

In [9]:
# | output: true

c = (
    alt.Chart(
        storage_provider_power_df.groupby("date")
        .sum()["quality_adjusted_power_pibs"]
        .reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("quality_adjusted_power_pibs:Q",
                title="Quality Adjusted PiB Power"),
        tooltip=["date", "quality_adjusted_power_pibs"]
    )
    .properties(width="container")
)

c.display()


alt.Chart(...)

In [10]:
# | output: true

display(Markdown(f"### Storage Providers With Power over Time"))

c = (
    alt.Chart(
        storage_provider_power_df[storage_provider_power_df["raw_power_pibs"] > 0]
        .groupby("date")
        .agg({"provider_id": "nunique"})
        .reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("provider_id:Q", title="Unique Providers"),
        tooltip=["date", "provider_id"]
    )
    .properties(width="container")
)

c.display()


### Storage Providers With Power over Time

alt.Chart(...)

:::

::: {layout="[ [1,1], [1] ]"}

### Storage Provider Raw PiB Power Growth Rate (7 Days Rolling Mean)

In [11]:
# | output: true

daily_pib_power = (
    storage_provider_power_df.groupby("date").sum()[
        "raw_power_pibs"].reset_index()
)

daily_pib_power = daily_pib_power.assign(
    growth_rate=lambda df: df["raw_power_pibs"].pct_change(
        periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_pib_power)
    .mark_line()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Raw PiB Power Growth Rate"),
        tooltip=["date", "growth_rate"],
    )
    .properties(width="container")
    # .interactive(bind_y=False)
)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


alt.LayerChart(...)

### Storage Providers With Power over Time Growth Rate (7 Days Rolling Mean)

In [12]:
# | output: true

daily_miners_with_power = (
    storage_provider_power_df[storage_provider_power_df["raw_power_pibs"] > 0]
    .groupby("date")
    .agg({"provider_id": "nunique"})
    .reset_index()
    .rename(columns={"date": "date", "provider_id": "unique_providers"})
)

daily_miners_with_power = daily_miners_with_power.assign(
    growth_rate=lambda df: df["unique_providers"].pct_change(
        periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_miners_with_power)
    .mark_line()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Unique Miners Growth Rate"),
        tooltip=["date", "growth_rate"]
    )
    .properties(width="container")
    # .interactive(bind_y=False)
)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


alt.LayerChart(...)

:::